In [1]:
# Uncomment this line if you need to install these packages:
#import Pkg; Pkg.add("JuMP"); Pkg.add("HiGHS"); Pkg.add("DataFrames"); Pkg.add("CSV"); Pkg.add("Statistics"); Pkg.add("Plots"); Pkg.add("SplitApplyCombine")
using JuMP, HiGHS
using Plots
using DataFrames, CSV, Statistics
using SplitApplyCombine

ENV["COLUMNS"]=120 # Set so all columns of DataFrames and Matrices are displayed

120

**1. Load parameters (data)**



In [2]:
using DataFrames
using CSV

# Load initial data from CSV file

# Generators contains cost info about each generator type
generators = DataFrame(CSV.File("Japan_Data/Generators.csv"))
    
# Flows contains info about where each line goes from and to and its exisiting capacity 
Flows = DataFrame(CSV.File("Japan_Data/Flows.csv"))
    
# Demand data for each node
Demand9Reg = DataFrame(CSV.File("Japan_DATA/Demand_9Regions.csv"));

# Solar Variability (capacity factor per hour for every node)
SolarVar = DataFrame(CSV.File("Japan_DATA/Solar_Var.csv"));

# Wind Variability (capacity factor per hour for every node)
WindVar = DataFrame(CSV.File("Japan_DATA/Wind_Var.csv"));

# Existing generator capacity
ExistingCap = DataFrame(CSV.File("Japan_DATA/Existing_CAP.csv"));


**2. Set Parameters**

In [3]:
NSECost = 9000;
flow_cost = 50;
FC_build_cost = 70000000;
FC_Var = 1000;

**3. Define sets**

In [4]:
# The set of hours 

H = collect(1:8760);

# The set of generators from the generators DataFrame
G = generators.G[1:(size(generators,1))];

#set f generators for which there are existing plants 
NEW = [2;3;4;5]; #add no new geothermal
RETI = [4]; #only type of generation that can be retired is thermal 
 
REG = [1;2;3];# The set of renewable 
#set of frequency converters
F = [1;2;3];

Fconv = [400;400;400];

# The set of nodes from the demand data
N = collect(1:ncol(Demand9Reg));

# The set of Lines
L = Flows.L;
P = Flows.Price_new_MW;
# Data on which nodes the lines are between
DIR_C = DataFrame(From = Flows.Node_From, To = Flows.Node_To)
# Data on the direction of the lines 
DIR_FLOW = zeros(length(N), length(L))
for l in L
    for n in N
        if DIR_C[l,"From"] == n 
            DIR_FLOW[n,l] = -1
        elseif DIR_C[l,"To"] == n
            DIR_FLOW[n,l] = 1
        end   
    end     
end


**4. Define the model and specify solver**

In [5]:
Expansion_Model = Model(HiGHS.Optimizer) # using the HiGHS open source solver

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

**5. Define design variables**

In [6]:
@variables(Expansion_Model, begin
        NSE[n in N, h in H] >= 0         # Non-served energy in each hour (MWh)
        CAP[g in NEW, n in N] >=0          # New capacity built for each generator type at each node  (MW)
        RET[g in G, n in N] >=0        # Capacity to be retired at each node only thermal can be retired
        GEN[g in G, n in N, h in H] >= 0 # Generation at each node for each generator in each hour (MWh)                
        FLOW[l in L, h in H]             # Flow on each line in each hour (MWh)
        FCAP[l in L]       >=0              # New Capacity of each line (MW)      
        FCON[f in F]       >=0                  # new frequency converter (MW)
end)


(2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9]
    Dimension 2, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  991, 992, 993, 994, 995, 996, 997, 998, 999, 1000]
And data, a 9×1000 Matrix{VariableRef}:
 NSE[1,1]  NSE[1,2]  NSE[1,3]  NSE[1,4]  NSE[1,5]  NSE[1,6]  …  NSE[1,997]  NSE[1,998]  NSE[1,999]  NSE[1,1000]
 NSE[2,1]  NSE[2,2]  NSE[2,3]  NSE[2,4]  NSE[2,5]  NSE[2,6]     NSE[2,997]  NSE[2,998]  NSE[2,999]  NSE[2,1000]
 NSE[3,1]  NSE[3,2]  NSE[3,3]  NSE[3,4]  NSE[3,5]  NSE[3,6]     NSE[3,997]  NSE[3,998]  NSE[3,999]  NSE[3,1000]
 NSE[4,1]  NSE[4,2]  NSE[4,3]  NSE[4,4]  NSE[4,5]  NSE[4,6]     NSE[4,997]  NSE[4,998]  NSE[4,999]  NSE[4,1000]
 NSE[5,1]  NSE[5,2]  NSE[5,3]  NSE[5,4]  NSE[5,5]  NSE[5,6]     NSE[5,997]  NSE[5,998]  NSE[5,999]  NSE[5,1000]
 NSE[6,1]  NSE[6,2]  NSE[6,3]  NSE[6,4]  NSE[6,5]  NSE[6,6]  …  NSE[6,997]  NSE[6,998]  NSE[6,999]  NSE[6,1000]
 NSE[7,1]  NSE[7,2]  NSE[7,3]  NSE[7,4]  NSE[7,5]  NSE[7,6]     NSE[7,997]

**6. Define constraints**

In [7]:
@constraints(Expansion_Model, begin
    cDemandBalance[h in H, n in N],  sum(GEN[g,n,h] for g in G) + sum(DIR_FLOW[n,l]*FLOW[l,h] for l in L) + NSE[n,h] == Demand9Reg[h,n] ;# Energy generated - flow out + flow in + NSE = demand every hour every node
    cThermalCapacity[g in RETI, n in N, h in H], GEN[g,n,h] <= CAP[g,n] + ExistingCap[g,n] - RET[g,n]; # generation smaller than capacity every hour every generator type every node   
    cWindCapacity[n in N, h in H], GEN[2,n,h] <= (CAP[2,n]*WindVar[h,n]) + (ExistingCap[2,n]*WindVar[h,n]);
    cSolarCapacity[n in N, h in H], GEN[3,n,h] <= (CAP[3,n]*SolarVar[h,n]) + (ExistingCap[3,n]*SolarVar[h,n]);    
    cNuclearCapacity[n in N, h in H], GEN[5,n,h] <= CAP[5,n] + ExistingCap[5,n]; 
    cNuclearRampP[n in N, h in H[2:end]], GEN[5,n,h]  <=  GEN[5,n,h-1] + (0.6*(CAP[5,n]+ExistingCap[5,n]));
    cNuclearRampN[n in N, h in H[2:end]], GEN[5,n,h] >=  GEN[5,n,h-1] - (0.6*(CAP[5,n]+ExistingCap[5,n]));
    cGeothermalCapacity[n in N, h in H], GEN[1,n,h] <= ExistingCap[1,n];  
    cNuclearMix, sum(GEN[5,n,h] for n in N, h in H) >= 0.21 * sum(GEN[g,n,h] for g in G, n in N, h in H); # Energy mix - Nuclear 21%   
    cRenewableMix, sum(GEN[g,n,h] for g in REG, n in N, h in H) >= 0.38 * sum(GEN[g,n,h] for g in G, n in N, h in H); # Energy mix - Renewables 38%,  Thermal 41%, Nuclear 21%   
    cFlowN[l in L, h in H], -(FCAP[l] + Flows[l,"Existing_Cap_MW"]) <= FLOW[l,h]; # Negative flow on each line must be less than capacity of that line for every hour 
    cFlowP[l in L, h in H],  FLOW[l,h] <= FCAP[l] + Flows[l,"Existing_Cap_MW"]; # Positive flow on each line must be less than capacity of that line for every hour 
    cFC[h in H], FLOW[3,h] <= sum(Fconv) + sum(FCON[f] for f in F);
    cFCCAPL[f in F], FCON[f] <= 400;
    #cFCCAPS[f in F], FCON[f] >= 399;
    
end)
    

(2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  991, 992, 993, 994, 995, 996, 997, 998, 999, 1000]
    Dimension 2, [1, 2, 3, 4, 5, 6, 7, 8, 9]
And data, a 1000×9 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 cDemandBalance[1,1] : NSE[1,1] + GEN[1,1,1] + GEN[2,1,1] + GEN[3,1,1] + GEN[4,1,1] + GEN[5,1,1] - FLOW[1,1] == 3285                          …  cDemandBalance[1,9] : NSE[9,1] + GEN[1,9,1] + GEN[2,9,1] + GEN[3,9,1] + GEN[4,9,1] + GEN[5,9,1] + FLOW[7,1] - FLOW[9,1] == 2420
 cDemandBalance[2,1] : NSE[1,2] + GEN[1,1,2] + GEN[2,1,2] + GEN[3,1,2] + GEN[4,1,2] + GEN[5,1,2] - FLOW[1,2] == 3439                             cDemandBalance[2,9] : NSE[9,2] + GEN[1,9,2] + GEN[

**7. Define objective function**

In [8]:
@objective(Expansion_Model, Min,
    sum(generators[generators.G.==g,:FixedCost][1]*CAP[g,n] for g in NEW, n in N) + 
    sum(generators[generators.G.==1,:FixedOM][1]*(ExistingCap[1,n]) for n in N) + # geo
    sum(generators[generators.G.==g,:FixedOM][1]*(ExistingCap[g,n]+CAP[g,n]) for g in NEW, n in N) + # thermal, wind, solar, nuclear
    sum(generators[generators.G.==g,:FixedOM][1]*(ExistingCap[g,n]+CAP[g,n]-RET[g,n]) for g in RETI, n in N) + # thermal
    sum(generators[generators.G.==g,:VarCost][1]*GEN[g,n,h] for  g in G, h in H, n in N) +
    sum(FCAP[l] * P[l] for l in L) +
    sum(FLOW[l,h]*flow_cost for l in L, h in H) +
    sum(NSECost*NSE[n,h] for h in H,  n in N)  +
    sum(FCON[f]* FC_build_cost for f in F) +
    sum((FCON[f]+sum(Fconv))* FC_Var for f in F)
);

**8. Solve the model**

In [9]:
Expansion_Model

A JuMP Model
Minimization problem with:
Variables: 64094
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 9000 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 8993 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 74994 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 54094 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: CAP, FCAP, FCON, FLOW, GEN, NSE, RET, cDemandBalance, cFC, cFCCAPL, cFlowN, cFlowP, cGeothermalCapacity, cNuclearCapacity, cNuclearMix, cNuclearRampN, cNuclearRampP, cRenewableMix, cSolarCapacity, cThermalCapacity, cWindCapacity

**9. Optimize model**

In [10]:
optimize!(Expansion_Model)

Running HiGHS 1.7.0 (git hash: 50670fd4c): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-04, 1e+00]
  Cost   [2e+00, 7e+07]
  Bound  [0e+00, 0e+00]
  RHS    [3e-02, 4e+04]
Presolving model
79991 rows, 55065 cols, 307981 nonzeros  0s
79991 rows, 55063 cols, 305981 nonzeros  0s
Presolve : Reductions: rows 79991(-12996); columns 55063(-9031); elements 305981(-41975)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -9.8519179077e+08 Ph1: 36079(2.89305e+07); Du: 10009(4.02394e+06) 0s
      18158     1.3992139547e+10 Pr: 9138(7.48173e+07); Du: 0(1.18266e-07) 5s
      21751     1.3997598075e+10 Pr: 8906(8.84035e+07); Du: 0(2.67637e-07) 11s
      25071     1.4413326385e+10 Pr: 8899(6.3384e+07); Du: 0(1.97005e-07) 16s
      28688     1.5294287163e+10 Pr: 8891(8.89188e+07); Du: 0(1.18266e-07) 21s
      34289     2.8683389945e+10 Pr: 9964(5.57426e+07); Du: 0(1.92164e-07) 26s

**10. Present results**

In [11]:
# out_GEN=value.(GEN).data;
# out_FLOW=value.(FLOW).data
# out_NSE=value.(NSE).data
# out_CAP=value.(CAP).data
# out_FCAP=value.(FCAP).data
# out_RET=value.(RET).data



out_GEN=value.(GEN).data;
out_FLOW=value.(FLOW).data
out_NSE=value.(NSE).data
out_CAP=value.(CAP).data
out_FCAP=value.(FCAP).data
out_RET=value.(RET).data
out_FC=value.(FCON).data
output_node = 7;
output_hour = 24;
sum_gen = sum(out_GEN[g,n,output_hour] for g in G, n in N);
total_gen_all_hours = sum(out_GEN[g,n,h] for g in G, n in N, h in H);
total_gen_renewable = sum(out_GEN[g,n,h] for g in G[2:3], n in N, h in H);
total_gen_nuclear = sum(out_GEN[g,n,h] for g in G[5], n in N, h in H);
# Print the results. Create a variable of node and hours that I can set. 

println("Generation at node ", output_node, " at hour ", output_hour, " of thermal is ", out_GEN[4,output_node,output_hour], "MW")
println("Generation at node ", output_node, " at hour ", output_hour, " of wind is ", out_GEN[2,output_node,output_hour], "MW")
println("Generation at node ", output_node, " at hour ", output_hour, " of solar is ", out_GEN[3,output_node,output_hour], "MW")
println("Generation at node ", output_node, " at hour ", output_hour, " of nuclear is ", out_GEN[5,output_node,output_hour], "MW")
# Total generation of all nodes at hour 50
println("Total generation at hour ", output_hour, " is ", sum_gen, "MW")
# Total generation at all nodes for all hours
println("Total generation for all hours is ", total_gen_all_hours, "MW")
# at this hour, the renewnable mix is  
println("Renewable mix at hour ", output_hour, " is ", 100*sum(out_GEN[g,n,output_hour] for g in G[2:3], n in N)/sum_gen, "%")
# at this hour, the nuclear mix is
println("Nuclear mix at hour ", output_hour, " is ",100*sum(out_GEN[G[5],n,output_hour] for n in N)/sum_gen, "%")
# Renewable mix for all hours
println("Renewable mix for all hours is ", 100*total_gen_renewable/total_gen_all_hours, "%")
# Nuclear mix for all hours
println("Nuclear mix for all hours is ", 100*total_gen_nuclear/total_gen_all_hours, "%")
#println(" Freq Conv", sum(out_FC[f] for f in F) )
# Now doing the same for the flows
output_line = 3;
println("Flow on line ", output_line, " at hour ", output_hour, " is ", out_FLOW[output_line,output_hour], "MW", " from node ", DIR_C[output_line,"From"], " to node ", DIR_C[output_line,"To"])
println(" New Flow on line ", output_line, " at hour ", output_hour, " is ", out_FCAP[output_line], "MW", " from node ", DIR_C[output_line,"From"], " to node ", DIR_C[output_line,"To"])

# println(out_GEN[2,4,50])
# println(out_GEN[3,4,50])
# println(out_GEN[4,4,50])
# println(out_FLOW[:,50])
 println(out_FCAP)
 println(out_FC)
# println(out_NSE[7,50])
# println(Demand9Reg[50, 7])


# println(ExistingCap[1,7])
# println(out_GEN[1,7,50])
# println(out_CAP[1,7])

   











Generation at node 7 at hour 24 of thermal is 1138.0MW
Generation at node 7 at hour 24 of wind is 192.0MW
Generation at node 7 at hour 24 of solar is -0.0MW
Generation at node 7 at hour 24 of nuclear is 167.0MW
Total generation at hour 24 is 94118.99999999999MW
Total generation for all hours is 8.539429483398604e7MW
Renewable mix at hour 24 is 47.47633716154067%
Nuclear mix at hour 24 is 24.368526257151014%
Renewable mix for all hours is 37.62861394768802%
Nuclear mix for all hours is 25.69732310234694%
Flow on line 3 at hour 24 is -1200.0MW from node 2 to node 3
 New Flow on line 3 at hour 24 is -0.0MW from node 2 to node 3
[0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1537.6136975450008]
[0.0, 0.0, 0.0]
